In [18]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import mstats
import os

In [ ]:
# --- Step 1: Load Data ---
file_path = os.getcwd() + '/data/Factor_data.xlsx'
df = pd.read_excel(file_path, sheet_name='dc_fund', index_col=0)
df = df.tail(36).copy()

"""
# Winsorize and standardize factors
for col in ['Quality', 'Momentum', 'Value', 'Low_vol', 'Market']:
    df[col] = mstats.winsorize(df[col], limits=[0.05, 0.05])
    df[col] = (df[col] - df[col].mean()) / df[col].std()
"""

"\n# Winsorize and standardize factors\nfor col in ['Quality', 'Momentum', 'Value', 'Low_vol', 'Market']:\n    df[col] = mstats.winsorize(df[col], limits=[0.05, 0.05])\n    df[col] = (df[col] - df[col].mean()) / df[col].std()\n"

In [20]:
market_neutral_factors = {}

for factor in ['Quality', 'Momentum', 'Value', 'Low_vol']:
    # Prepare regression variables
    x_market = sm.add_constant(df['Market'])
    y_factor = df[factor]

    
    # Run regression
    model = sm.OLS(y_factor, x_market).fit()
    
    # Get residuals (market-neutral factor returns)
    market_neutral_factors[factor] = model.resid

# Add market-neutral factors to DataFrame
for factor in market_neutral_factors:
    df[f'{factor}_mn'] = market_neutral_factors[factor]

# Now use ['Quality_mn', 'Momentum_mn', 'Value_mn', 'Low_vol_mn'] as your factors in further analysis

print(df.columns)

Index(['Net Return', 'Benchmark Return', 'Quality', 'Momentum', 'Value',
       'Low_vol', 'Market', 'Quality_mn', 'Momentum_mn', 'Value_mn',
       'Low_vol_mn'],
      dtype='object')


In [31]:
df[['Quality', 'Momentum', 'Value', 'Low_vol', 'Market']].corr()

,Quality,Momentum,Value,Low_vol,Market
Quality,1.000000,0.896456,0.835524,0.865622,0.908279
Momentum,0.896456,1.000000,0.868006,0.838115,0.923693
Value,0.835524,0.868006,1.000000,0.840170,0.865474
Low_vol,0.865622,0.838115,0.840170,1.000000,0.929970
Market,0.908279,0.923693,0.865474,0.929970,1.000000


In [29]:
df[['Quality_mn', 'Momentum_mn', 'Value_mn', 'Low_vol_mn', 'Market']].corr()

,Quality_mn,Momentum_mn,Value_mn,Low_vol_mn,Market
Quality_mn,1.000000e+00,3.586336e-01,2.358629e-01,1.362120e-01,4.796953e-16
Momentum_mn,3.586336e-01,1.000000e+00,3.572802e-01,-1.483210e-01,5.140398e-16
Value_mn,2.358629e-01,3.572802e-01,1.000000e+00,1.917020e-01,4.649157e-16
Low_vol_mn,1.362120e-01,-1.483210e-01,1.917020e-01,1.000000e+00,2.627379e-16
Market,4.796953e-16,5.140398e-16,4.649157e-16,2.627379e-16,1.000000e+00


In [21]:
# --- Step 3: Prepare Regression Variables ---
X = df[['Quality_mn', 'Momentum_mn', 'Value_mn', 'Low_vol_mn', 'Market']]
# X = df[['Quality', 'Momentum', 'Value', 'Low_vol']] 
y = df['Net Return']  # or df['Benchmark Return']

# Add constant for intercept
X = sm.add_constant(X)

# --- Step 4: Run Regression ---
model = sm.OLS(y, X).fit()
print(model.summary())

# --- Step 5: Get Betas (Factor Sensitivities) ---
betas = model.params.drop('const')
print(betas)

                            OLS Regression Results                            
Dep. Variable:             Net Return   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     1.901
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.132
Time:                        00:18:30   Log-Likelihood:                 109.61
No. Observations:                  30   AIC:                            -207.2
Df Residuals:                      24   BIC:                            -198.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0093      0.001      6.799      

In [22]:
# --- Step 6: Calculate Factor Variances and Covariances ---
factor_returns = df[['Quality_mn', 'Momentum_mn', 'Value_mn', 'Low_vol_mn', 'Market']]
#factor_returns = df[['Quality', 'Momentum', 'Value', 'Low_vol']] 
factor_cov = factor_returns.cov()
factor_var = factor_returns.var()

factor_cov

,Quality_mn,Momentum_mn,Value_mn,Low_vol_mn,Market
Quality_mn,4.123953e-04,1.613065e-04,1.346749e-04,3.648162e-05,3.738628e-19
Momentum_mn,1.613065e-04,4.905553e-04,2.224966e-04,-4.332599e-05,5.021445e-19
Value_mn,1.346749e-04,2.224966e-04,7.905709e-04,7.108843e-05,4.897603e-19
Low_vol_mn,3.648162e-05,-4.332599e-05,7.108843e-05,1.739416e-04,1.644996e-19
Market,3.738628e-19,5.021445e-19,4.897603e-19,1.644996e-19,1.465643e-03


In [23]:
# --- Step 7: Calculate Risk Attribution ---
# Portfolio variance explained by factors
risk_contributions = {}
for i, factor in enumerate(factor_returns.columns):
    # Variance term
    var_term = betas[factor]**2 * factor_var[factor]
    # Covariance terms
    cov_term = 0
    for j, other_factor in enumerate(factor_returns.columns):
        if i != j:
            cov_term += betas[factor] * betas[other_factor] * factor_cov.loc[factor, other_factor]
    # Total contribution
    risk_contributions[factor] = var_term + cov_term

# Residual (unexplained) risk
residual_var = model.resid.var()

In [24]:
# --- Step 8: Summarize Results ---
total_risk = sum(risk_contributions.values()) + residual_var

print("\nRisk Attribution (Absolute):")
for factor, contribution in risk_contributions.items():
    print(f"{factor}: {contribution:.6f} ({contribution/total_risk:.2%} of total risk)")
print(f"Residual: {residual_var:.6f} ({residual_var/total_risk:.2%} of total risk)")
print(f"Total Portfolio Variance: {total_risk:.6f}")


Risk Attribution (Absolute):
Quality_mn: 0.000000 (0.33% of total risk)
Momentum_mn: -0.000000 (-0.39% of total risk)
Value_mn: 0.000009 (15.26% of total risk)
Low_vol_mn: 0.000002 (3.57% of total risk)
Market: 0.000005 (9.60% of total risk)
Residual: 0.000041 (71.63% of total risk)
Total Portfolio Variance: 0.000057


In [25]:
# --- Step 9: Attribution Calculation ---
betas = model.params.drop('const')
alpha = model.params['const']
avg_factors = X.drop('const', axis=1).mean()

# Factor contributions
factor_contributions = betas * avg_factors

# Alpha contribution
alpha_contribution = alpha

# Residual (unexplained) contribution
residuals = model.resid
residual_contribution = residuals.mean()

# Total (should match average net return)
total_contribution = factor_contributions.sum() + alpha_contribution + residual_contribution

# --- Step 6: Display Results ---
print("Return Attribution (last 36 rows):")
print(f"Alpha: {alpha_contribution:.6%}")
for factor in betas.index:
    print(f"{factor}: {factor_contributions[factor]:.6%}")
print(f"Residual: {residual_contribution:.6%}")
print(f"Total: {total_contribution:.6%} (should match average Net Return: {y.mean():.6%})")

Return Attribution (last 36 rows):
Alpha: 0.928872%
Quality_mn: 0.000000%
Momentum_mn: 0.000000%
Value_mn: -0.000000%
Low_vol_mn: -0.000000%
Market: 0.086211%
Residual: 0.000000%
Total: 1.015083% (should match average Net Return: 1.015083%)


,Quality,Momentum,Value,Low_vol
Quality,1.000000,0.896456,0.835524,0.865622
Momentum,0.896456,1.000000,0.868006,0.838115
Value,0.835524,0.868006,1.000000,0.840170
Low_vol,0.865622,0.838115,0.840170,1.000000


,Quality_mn,Momentum_mn,Value_mn,Low_vol_mn,Market
Quality_mn,1.000000e+00,3.586336e-01,2.358629e-01,1.362120e-01,4.796953e-16
Momentum_mn,3.586336e-01,1.000000e+00,3.572802e-01,-1.483210e-01,5.140398e-16
Value_mn,2.358629e-01,3.572802e-01,1.000000e+00,1.917020e-01,4.649157e-16
Low_vol_mn,1.362120e-01,-1.483210e-01,1.917020e-01,1.000000e+00,2.627379e-16
Market,4.796953e-16,5.140398e-16,4.649157e-16,2.627379e-16,1.000000e+00
